<a href="https://colab.research.google.com/github/ayulockin/DLshots/blob/master/Batch_Normalization_Effect_of_Batch_Size.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setups, Installations and Imports

In [ ]:
!nvidia-smi

Sat Jun 27 16:10:47 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%capture
!pip install wandb

In [ ]:
SEED = 666

import tensorflow as tf
tf.random.set_seed(SEED)

from tensorflow import keras
from tensorflow.keras.datasets import cifar10

In [ ]:
import os
os.environ["TF_DETERMINISTIC_OPS"] = "1"

import numpy as np
np.random.seed(SEED)

import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

from mpl_toolkits.axes_grid1 import ImageGrid
from PIL import Image

In [ ]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

<IPython.core.display.Javascript object>

wandb: ERROR Not authenticated.  Copy a key from https://app.wandb.ai/authorize


API Key: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Download and Prepare Dataset

#### CIFAR-10

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 6s 0us/step


In [ ]:
CLASS_NAMES = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

In [ ]:
print('Shape of x_train: ', x_train.shape)
print('Shape of y_train: ', y_train.shape)
print('Shape of x_test: ', x_test.shape)
print('Shape of y_test: ', y_test.shape)

Shape of x_train:  (50000, 32, 32, 3)
Shape of y_train:  (50000, 1)
Shape of x_test:  (10000, 32, 32, 3)
Shape of y_test:  (10000, 1)


# Dataloader Utils

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
IMG_SHAPE = 32
CHANNELS = 3
NUM_CLASSES = 10

In [ ]:
def preprocess_image(image, label):
  image = image/255
  image = tf.image.convert_image_dtype(image, tf.float32)
  return image, label

# Callbacks

In [ ]:
def scheduler(epoch):
  if epoch < 7:
    return 0.001
  else:
    return 0.001 * tf.math.exp(0.1 * (7 - epoch))

lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

# Model

In [ ]:
def ModelBN():
  inputs = keras.layers.Input(shape=(IMG_SHAPE, IMG_SHAPE, CHANNELS))

  x = keras.layers.Conv2D(filters=32,
                          kernel_size=(3,3),
                          strides=(1,1),
                          padding='valid')(inputs)
  x = keras.layers.BatchNormalization()(x)
  x = keras.activations.relu(x)

  x = keras.layers.Conv2D(filters=32, 
                          kernel_size=(3,3),
                          strides=(1,1),
                          padding='valid')(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.activations.relu(x)

  x = keras.layers.MaxPooling2D(pool_size=2)(x)

  x = keras.layers.Conv2D(filters=32, 
                          kernel_size=(3,3),
                          strides=(1,1),
                          padding='valid')(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.activations.relu(x)

  x = keras.layers.Conv2D(filters=32, 
                          kernel_size=(3,3),
                          strides=(1,1),
                          padding='valid')(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.activations.relu(x)

  x = keras.layers.GlobalAveragePooling2D()(x)

  x = keras.layers.Dense(128, activation='relu')(x)
  x = keras.layers.Dense(32, activation='relu')(x)
  
  outputs = keras.layers.Dense(NUM_CLASSES, activation='softmax')(x)

  return keras.models.Model(inputs=inputs, outputs=outputs)

# Sweep Train 

In [ ]:
def train():
    # Initialize wandb with a sample project name
    wandb.init()
    
    # Specify the hyperparameter to be tuned along with
    # an initial value
    configs = {
        'batch_size': 8
    }

    # Specify the other hyperparameters to the configuration
    config = wandb.config
    config.epochs = 25

    # Add the config item (layers) to wandb
    if wandb.run:
        wandb.config.update({k: v for k, v in configs.items() if k not in dict(wandb.config.user_items())})
        configs = dict(wandb.config.user_items())

    # Prepare data loader
    trainloader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    testloader = tf.data.Dataset.from_tensor_slices((x_test, y_test))

    trainloader = (
        trainloader
        .shuffle(1024)
        .map(preprocess_image, num_parallel_calls=AUTO)
        .batch(wandb.config.batch_size)
        .prefetch(AUTO)
    )

    testloader = (
        testloader
        .map(preprocess_image, num_parallel_calls=AUTO)
        .batch(wandb.config.batch_size)
        .prefetch(AUTO)
    )
  
    # Iniialize model
    keras.backend.clear_session()
    model = ModelBN()
    
    # Compile the model
    model.compile('adam', 'sparse_categorical_crossentropy', metrics=['acc'])
    
    # Train the model
    _ = model.fit(trainloader,
                  epochs=config.epochs,
                  validation_data=testloader,
                  callbacks=[WandbCallback(),
                            lr_callback])

    # Evaluate    
    loss, accuracy = model.evaluate(testloader, callbacks=[WandbCallback()])
    print('Test Error Rate: ', round((1-accuracy)*100, 2))
    wandb.log({'Test Error Rate': round((1-accuracy)*100, 2)})

In [ ]:
sweep_config = {
  'method': 'grid',
  'parameters': {
      'batch_size': {
          'values': [8, 16, 32, 64, 128, 256, 512, 1024, 2048]
      }
  }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity='authors', project="seo")

Create sweep with ID: 2w36r6w1
Sweep URL: https://app.wandb.ai/authors/seo/sweeps/2w36r6w1


In [ ]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: 5ldej7vv with config:
	batch_size: 8
wandb: Agent Started Run: 5ldej7vv


Epoch 1/25
6250/6250 [==============================] - 60s 10ms/step - loss: 1.5241 - acc: 0.4424 - val_loss: 1.3479 - val_acc: 0.5208 - lr: 0.0010
Epoch 2/25
6250/6250 [==============================] - 61s 10ms/step - loss: 1.1806 - acc: 0.5802 - val_loss: 1.1342 - val_acc: 0.5957 - lr: 0.0010
Epoch 3/25
6250/6250 [==============================] - 60s 10ms/step - loss: 1.0599 - acc: 0.6259 - val_loss: 1.1041 - val_acc: 0.6045 - lr: 0.0010
Epoch 4/25
6250/6250 [==============================] - 60s 10ms/step - loss: 0.9850 - acc: 0.6536 - val_loss: 1.0082 - val_acc: 0.6455 - lr: 0.0010
Epoch 5/25
6250/6250 [==============================] - 60s 10ms/step - loss: 0.9345 - acc: 0.6737 - val_loss: 1.0407 - val_acc: 0.6305 - lr: 0.0010
Epoch 6/25
6250/6250 [==============================] - 60s 10ms/step - loss: 0.8873 - acc: 0.6919 - val_loss: 0.8685 - val_acc: 0.6966 - lr: 0.0010
Epoch 7/25
6250/6250 [==============================] - 61s 10ms/step - loss: 0.8591 - acc: 0.7033 - val_l

Epoch 1/25
3125/3125 [==============================] - 32s 10ms/step - loss: 1.4305 - acc: 0.4758 - val_loss: 1.3895 - val_acc: 0.5138 - lr: 0.0010
Epoch 2/25
3125/3125 [==============================] - 32s 10ms/step - loss: 1.1032 - acc: 0.6076 - val_loss: 1.2194 - val_acc: 0.5721 - lr: 0.0010
Epoch 3/25
3125/3125 [==============================] - 32s 10ms/step - loss: 0.9928 - acc: 0.6479 - val_loss: 1.0806 - val_acc: 0.6268 - lr: 0.0010
Epoch 4/25
3125/3125 [==============================] - 32s 10ms/step - loss: 0.9131 - acc: 0.6785 - val_loss: 1.0378 - val_acc: 0.6233 - lr: 0.0010
Epoch 5/25
3125/3125 [==============================] - 32s 10ms/step - loss: 0.8634 - acc: 0.6958 - val_loss: 0.8996 - val_acc: 0.6862 - lr: 0.0010
Epoch 6/25
3125/3125 [==============================] - 32s 10ms/step - loss: 0.8148 - acc: 0.7154 - val_loss: 0.9295 - val_acc: 0.6774 - lr: 0.0010
Epoch 7/25
3125/3125 [==============================] - 32s 10ms/step - loss: 0.7866 - acc: 0.7255 - val_l

Epoch 1/25
1563/1563 [==============================] - 21s 13ms/step - loss: 1.4034 - acc: 0.4846 - val_loss: 1.5492 - val_acc: 0.4620 - lr: 0.0010
Epoch 2/25
1563/1563 [==============================] - 20s 13ms/step - loss: 1.0572 - acc: 0.6209 - val_loss: 1.0813 - val_acc: 0.6179 - lr: 0.0010
Epoch 3/25
1563/1563 [==============================] - 20s 13ms/step - loss: 0.9325 - acc: 0.6686 - val_loss: 1.3147 - val_acc: 0.5732 - lr: 0.0010
Epoch 4/25
1563/1563 [==============================] - 20s 13ms/step - loss: 0.8569 - acc: 0.6957 - val_loss: 1.1410 - val_acc: 0.5991 - lr: 0.0010
Epoch 5/25
1563/1563 [==============================] - 21s 13ms/step - loss: 0.8050 - acc: 0.7155 - val_loss: 0.8785 - val_acc: 0.6883 - lr: 0.0010
Epoch 6/25
1563/1563 [==============================] - 20s 13ms/step - loss: 0.7622 - acc: 0.7313 - val_loss: 0.8742 - val_acc: 0.6886 - lr: 0.0010
Epoch 7/25
1563/1563 [==============================] - 20s 13ms/step - loss: 0.7374 - acc: 0.7420 - val_l

Epoch 1/25
782/782 [==============================] - 14s 18ms/step - loss: 1.4114 - acc: 0.4823 - val_loss: 1.4319 - val_acc: 0.4996 - lr: 0.0010
Epoch 2/25
782/782 [==============================] - 14s 17ms/step - loss: 1.0576 - acc: 0.6231 - val_loss: 1.2672 - val_acc: 0.5477 - lr: 0.0010
Epoch 3/25
782/782 [==============================] - 13s 17ms/step - loss: 0.9343 - acc: 0.6676 - val_loss: 1.3124 - val_acc: 0.5604 - lr: 0.0010
Epoch 4/25
782/782 [==============================] - 13s 17ms/step - loss: 0.8519 - acc: 0.6969 - val_loss: 1.0187 - val_acc: 0.6379 - lr: 0.0010
Epoch 5/25
782/782 [==============================] - 13s 17ms/step - loss: 0.7966 - acc: 0.7185 - val_loss: 1.0565 - val_acc: 0.6351 - lr: 0.0010
Epoch 6/25
782/782 [==============================] - 13s 17ms/step - loss: 0.7543 - acc: 0.7357 - val_loss: 1.1117 - val_acc: 0.6301 - lr: 0.0010
Epoch 7/25
782/782 [==============================] - 13s 17ms/step - loss: 0.7152 - acc: 0.7479 - val_loss: 0.9636 - 

Epoch 1/25
391/391 [==============================] - 10s 27ms/step - loss: 1.4758 - acc: 0.4559 - val_loss: 2.3184 - val_acc: 0.2454 - lr: 0.0010
Epoch 2/25
391/391 [==============================] - 10s 26ms/step - loss: 1.0941 - acc: 0.6059 - val_loss: 1.2032 - val_acc: 0.5652 - lr: 0.0010
Epoch 3/25
391/391 [==============================] - 10s 25ms/step - loss: 0.9662 - acc: 0.6555 - val_loss: 1.2041 - val_acc: 0.5704 - lr: 0.0010
Epoch 4/25
391/391 [==============================] - 10s 25ms/step - loss: 0.8776 - acc: 0.6877 - val_loss: 0.9846 - val_acc: 0.6455 - lr: 0.0010
Epoch 5/25
391/391 [==============================] - 10s 24ms/step - loss: 0.8186 - acc: 0.7077 - val_loss: 1.0533 - val_acc: 0.6249 - lr: 0.0010
Epoch 6/25
391/391 [==============================] - 10s 25ms/step - loss: 0.7757 - acc: 0.7245 - val_loss: 1.1818 - val_acc: 0.6127 - lr: 0.0010
Epoch 7/25
391/391 [==============================] - 10s 25ms/step - loss: 0.7343 - acc: 0.7401 - val_loss: 0.9419 - 

Epoch 1/25
196/196 [==============================] - 9s 45ms/step - loss: 1.5796 - acc: 0.4213 - val_loss: 3.3767 - val_acc: 0.1001 - lr: 0.0010
Epoch 2/25
196/196 [==============================] - 8s 42ms/step - loss: 1.1833 - acc: 0.5736 - val_loss: 2.2974 - val_acc: 0.2736 - lr: 0.0010
Epoch 3/25
196/196 [==============================] - 8s 42ms/step - loss: 1.0366 - acc: 0.6299 - val_loss: 1.9642 - val_acc: 0.3944 - lr: 0.0010
Epoch 4/25
196/196 [==============================] - 8s 42ms/step - loss: 0.9458 - acc: 0.6641 - val_loss: 1.3164 - val_acc: 0.5328 - lr: 0.0010
Epoch 5/25
196/196 [==============================] - 8s 42ms/step - loss: 0.8792 - acc: 0.6891 - val_loss: 1.1608 - val_acc: 0.5839 - lr: 0.0010
Epoch 6/25
196/196 [==============================] - 8s 42ms/step - loss: 0.8337 - acc: 0.7065 - val_loss: 1.0265 - val_acc: 0.6395 - lr: 0.0010
Epoch 7/25
196/196 [==============================] - 8s 41ms/step - loss: 0.7889 - acc: 0.7231 - val_loss: 0.9377 - val_acc

Epoch 1/25
98/98 [==============================] - 8s 80ms/step - loss: 1.7253 - acc: 0.3741 - val_loss: 3.1914 - val_acc: 0.0997 - lr: 0.0010
Epoch 2/25
98/98 [==============================] - 7s 72ms/step - loss: 1.2796 - acc: 0.5346 - val_loss: 3.7289 - val_acc: 0.0999 - lr: 0.0010
Epoch 3/25
98/98 [==============================] - 7s 72ms/step - loss: 1.1252 - acc: 0.5952 - val_loss: 3.3402 - val_acc: 0.1131 - lr: 0.0010
Epoch 4/25
98/98 [==============================] - 7s 73ms/step - loss: 1.0284 - acc: 0.6341 - val_loss: 2.7371 - val_acc: 0.1960 - lr: 0.0010
Epoch 5/25
98/98 [==============================] - 7s 73ms/step - loss: 0.9574 - acc: 0.6597 - val_loss: 1.8590 - val_acc: 0.3776 - lr: 0.0010
Epoch 6/25
98/98 [==============================] - 7s 72ms/step - loss: 0.9065 - acc: 0.6792 - val_loss: 1.3288 - val_acc: 0.5209 - lr: 0.0010
Epoch 7/25
98/98 [==============================] - 7s 72ms/step - loss: 0.8657 - acc: 0.6948 - val_loss: 1.2610 - val_acc: 0.5682 - lr:

Epoch 1/25
49/49 [==============================] - 8s 156ms/step - loss: 1.9228 - acc: 0.3088 - val_loss: 2.5299 - val_acc: 0.1520 - lr: 0.0010
Epoch 2/25
49/49 [==============================] - 7s 133ms/step - loss: 1.4202 - acc: 0.4793 - val_loss: 3.2365 - val_acc: 0.1032 - lr: 0.0010
Epoch 3/25
49/49 [==============================] - 7s 133ms/step - loss: 1.2421 - acc: 0.5512 - val_loss: 3.7989 - val_acc: 0.1019 - lr: 0.0010
Epoch 4/25
49/49 [==============================] - 7s 133ms/step - loss: 1.1242 - acc: 0.5964 - val_loss: 4.3472 - val_acc: 0.1214 - lr: 0.0010
Epoch 5/25
49/49 [==============================] - 7s 133ms/step - loss: 1.0461 - acc: 0.6267 - val_loss: 4.5544 - val_acc: 0.1271 - lr: 0.0010
Epoch 6/25
49/49 [==============================] - 6s 132ms/step - loss: 0.9892 - acc: 0.6469 - val_loss: 4.1410 - val_acc: 0.1414 - lr: 0.0010
Epoch 7/25
49/49 [==============================] - 6s 132ms/step - loss: 0.9380 - acc: 0.6676 - val_loss: 4.0370 - val_acc: 0.155

Epoch 1/25
25/25 [==============================] - 7s 298ms/step - loss: 2.1034 - acc: 0.2620 - val_loss: 2.2987 - val_acc: 0.1086 - lr: 0.0010
Epoch 2/25
25/25 [==============================] - 6s 248ms/step - loss: 1.6825 - acc: 0.3767 - val_loss: 2.5897 - val_acc: 0.1332 - lr: 0.0010
Epoch 3/25
25/25 [==============================] - 6s 249ms/step - loss: 1.4315 - acc: 0.4742 - val_loss: 2.8739 - val_acc: 0.1003 - lr: 0.0010
Epoch 4/25
25/25 [==============================] - 6s 249ms/step - loss: 1.3028 - acc: 0.5281 - val_loss: 3.2928 - val_acc: 0.1000 - lr: 0.0010
Epoch 5/25
25/25 [==============================] - 6s 247ms/step - loss: 1.2145 - acc: 0.5635 - val_loss: 3.5515 - val_acc: 0.1003 - lr: 0.0010
Epoch 6/25
25/25 [==============================] - 6s 249ms/step - loss: 1.1451 - acc: 0.5912 - val_loss: 3.7018 - val_acc: 0.1045 - lr: 0.0010
Epoch 7/25
25/25 [==============================] - 6s 248ms/step - loss: 1.0893 - acc: 0.6099 - val_loss: 3.7768 - val_acc: 0.124